In [19]:
import math
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from bokeh.models import Legend,Plot,GraphRenderer, Oval, Range1d, MultiLine, Circle, HoverTool, TapTool, BoxSelectTool,BoxZoomTool, Rect, WheelZoomTool
from bokeh.models.graphs import NodesAndLinkedEdges, EdgesAndLinkedNodes,from_networkx
from bokeh.palettes import Spectral4, Spectral8, Blues8, Greens8, magma,OrRd,Oranges, Viridis256, Oranges8
from bokeh.plotting import ColumnDataSource
from bokeh.models import ColumnDataSource, LabelSet, ColumnData,StaticLayoutProvider,GraphRenderer
from bokeh.layouts import gridplot
import numpy as np
from networkx.algorithms import approximation as approx
import matplotlib.pyplot as plt
import pandas as pd
import networkx as nx
import numpy as np
import csv

In [20]:
#Importing Nodes as Pandas DF
nodes = pd.read_csv("sheep_data/sheep_age.csv")
edges = pd.read_csv("sheep_data/sheep_edges.csv")
#Importing Edges as Tuples 
with open('sheep_data/sheep_edges.csv') as f:
    sheep_edges=[tuple(line) for line in csv.reader(f)]
sheep_edges.pop(0)


('source', 'target', 'weight')

In [21]:
#Creating the Network
X = nx.DiGraph()
X.add_weighted_edges_from(sheep_edges)
#Creating Features
src_dom = edges.groupby(['source']).sum()
tgt_dom = edges.groupby(['target']).sum()
src_dom = src_dom.reset_index()
tgt_dom = tgt_dom.reset_index()
df2 = pd.DataFrame([[1, 0, 0]],columns=['target','source','weight'])
tgt_dom = tgt_dom.append(df2)
tgt_dom = tgt_dom.sort_values('target')
tgt_dom = tgt_dom.drop(columns=['source'])
tgt_dom.rename(columns={'target': 'id', 'weight':'DominatedBy'}, inplace=True)


In [22]:
df2 = pd.DataFrame([[6, 0, 0]],columns=['source', 'target','weight'])
src_dom = src_dom.append(df2)
src_dom = src_dom.sort_values('source')
src_dom = src_dom.drop(columns=['target'])
src_dom.rename(columns={'source': 'id', 'weight':'DominatedOthers'}, inplace=True)                                                                  

In [23]:
#Merge Original Nodes DF to add Dominated
nodes = pd.merge(nodes, tgt_dom, how='inner', on='id')
nodes = pd.merge(nodes, src_dom, how='inner', on='id')
#1 - How many different sheep it dominated
src_diff = edges.groupby(['source']).sum()
#2 - How many different sheep it was dominated by
tgt_diff = edges.groupby(['target']).sum()
edges_work = edges.copy()
edges_work = edges_work.drop(columns=['weight'])
#del src
src = edges_work.groupby(['source']).count()
src = src.reset_index()
df2 = pd.DataFrame([[6, 0]],columns=['source', 'target'])
src = src.append(df2)
src = src.sort_values('source')
src.rename(columns={'source': 'id', 'target':'TargetsPerSource'}, inplace=True)
edges_work = edges.copy()
edges_work = edges_work.drop(columns=['weight'])
#del tgt
tgt = edges_work.groupby('target').count()
tgt = tgt.reset_index()
df2 = pd.DataFrame([[1, 0]],columns=['target', 'source'])
tgt = tgt.append(df2)
tgt = tgt.sort_values('target')
tgt.rename(columns={'target': 'id', 'source':'SourcesPerTarget'}, inplace=True)
nodes = pd.merge(nodes, src, how='inner', on='id')
nodes = pd.merge(nodes, tgt, how='inner', on='id')
#Load the node attributes
for j in nodes.columns:
    for i,k in zip(sorted(X.nodes()),pd.Series(nodes[j])):
        X.node[i][j] = k
#Create Edge Attributes:
color_series = []
for i in edges['weight']:
    if i>7:
        color_series.append(Spectral8[7])
    else:
        color_series.append(Spectral8[i])
edges['weight_color'] = color_series


In [24]:
#nodes['DominatedOthers'] = nodes['DominatedOthers']+1
#nodes['DominatedBy'] = nodes['DominatedBy']+1
#nodes['TargetsPerSource'] = nodes['TargetsPerSource']+1
#nodes['SourcesPerTarget'] = nodes['SourcesPerTarget']+1
nodes['DomBy/DomOthers'] = (nodes['DominatedBy']+1).divide(nodes['DominatedOthers']+1)
#nodes['DomBy/DomOthers'] = nodes['DomBy/DomOthers']+.1
#nodes['Tgts/Srcs'] = (nodes['TargetsPerSource']+1).divide(nodes['SourcesPerTarget']+1)
nodes['DominatedOthersPCGT'] = (nodes['DominatedOthers']+1)/nodes['DominatedOthers'].sum()
#nodes['DominatedByPCGT'] = (nodes['DominatedBy']+1)/nodes['DominatedBy'].sum()
nodes['TotalInteractions'] = nodes['DominatedOthers']+nodes['DominatedBy']
nodes['WonPCGT'] = (nodes['DominatedOthers']+1)/nodes['TotalInteractions']
nds_dom_pt_lst = []
for i in nodes['DominatedOthersPCGT'].tolist():
    nds_dom_pt_lst.append(str(i))

In [25]:
nodes['tot_unique_interactions'] = nodes['TargetsPerSource']+nodes['SourcesPerTarget']


In [26]:
nodes.head(2)

,id,age,DominatedBy,DominatedOthers,TargetsPerSource,SourcesPerTarget,DomBy/DomOthers,DominatedOthersPCGT,TotalInteractions,WonPCGT,tot_unique_interactions
0,1,8,0,5,5,0,0.166667,0.009119,5,1.200000,5
1,2,8,9,35,14,5,0.277778,0.054711,44,0.818182,19


In [27]:
color_series2 = []
for i in nodes['WonPCGT']:
    if i<0.2:
        color_series2.append(Oranges8[7])
    elif i<0.3:
        color_series2.append(Oranges8[6])
    elif i<0.4:
        color_series2.append(Oranges8[5])
    elif i<0.5:
        color_series2.append(Oranges8[4])
    elif i<0.6:
        color_series2.append(Oranges8[3])
    elif i<0.7:
        color_series2.append(Oranges8[2])
    elif i<0.8:
        color_series2.append(Oranges8[1])
    else:
        color_series2.append(Oranges8[0])
nodes['WonColor'] = color_series2


In [28]:
color_series3 = []
for i in 1/nodes['DomBy/DomOthers']:
    if i<0.5:
        color_series3.append(Blues8[7])
    elif i<1:
        color_series3.append(Blues8[6])
    elif i<1.5:
        color_series3.append(Blues8[5])
    elif i<2:
        color_series3.append(Blues8[4])
    elif i<2.5:
        color_series3.append(Blues8[3])
    elif i<3:
        color_series3.append(Blues8[2])
    elif i<3.5:
        color_series3.append(Blues8[1])
    else:
        color_series3.append(Blues8[0])
nodes['DomBy/DomOthers_color'] = color_series3


In [29]:
color_series4 = []
for i in nodes['tot_unique_interactions']:
    if i<6:
        color_series4.append(Greens8[0])
    elif i<9:
        color_series4.append(Greens8[1])
    elif i<13:
        color_series4.append(Greens8[2])
    elif i<16:
        color_series4.append(Greens8[3])
    elif i<18:
        color_series4.append(Greens8[4])
    elif i<19:
        color_series4.append(Greens8[5])
    elif i<20:
        color_series4.append(Greens8[6])
    else:
        color_series4.append(Greens8[7])
nodes['tot_unique_interactions_color'] = color_series4


In [30]:
#First Plot
nds = len(X.nodes)
nd_idx = [i+1 for i in (range(nds))]

s1 = figure(title="Step1: Spectral Graph", x_range=(-1.1,1.1), y_range=(-1.1,1.1), 
            tools='pan,wheel_zoom,tap,xbox_select,reset', 
            toolbar_location="left", width=600, plot_height=600)

s1.title.text_color = "black"
s1.title.text_font = "verdana"
s1.title.text_font_style = "italic"
s1.title.text_font_style = "bold"
s1.background_fill_alpha = 0.07
s1.title.align = "center"
s1.title.text_font_size = '15pt'

s1.add_tools(HoverTool(tooltips=[("Source", "@edges_src"),
                                ("Target", "@edges_tgt"),("Weight","@width")]), 
            TapTool(), BoxSelectTool(), BoxZoomTool(),WheelZoomTool())

graph1 = from_networkx(X,nx.spectral_layout, center=(0,0))

pos = nx.spectral_layout(X)
x,y=zip(*pos.values())

src = ColumnDataSource({'x':x,'y':y,'nodeid':nd_idx})
labels1 = LabelSet(x='x', y='y', text='nodeid', source=src)


node_src = dict(index=nd_idx, dom_index = 30/nodes['DomBy/DomOthers'],
                                    dom_by = nodes['DominatedBy']/7,
                                    fi_color = nodes['DomBy/DomOthers_color'],
                                    fi_color2 = nodes['tot_unique_interactions_color'],
                                    fi_color3 = nodes['WonColor'],
                                    age = nodes['age']/60,
                                    tgtdivsrcstgt=nodes['tot_unique_interactions']*3,#tgtdivsrcstgt
                                    size_dim = nodes['DominatedOthersPCGT']*600)
                        
edge_src = dict(start=edges['source'],end=edges['target'],
                                     width=edges['weight'],
                                     edge_col=edges['weight_color'],
                                     width_alpha=edges['weight']/2,
                                     edges_src=edges['source'],
                                     edges_tgt=edges['target']) 

graph1.node_renderer.glyph = Circle(size="size_dim", fill_alpha="dom_index",
                                            line_color = "firebrick",
                                            line_dash = [1, 1],
                                            line_width = "dom_by",
                                            fill_color="fi_color")

graph1.node_renderer.selection_glyph = Circle(size="size_dim", fill_alpha="dom_index",
                                            line_color = "firebrick",
                                            line_dash = [1, 1],
                                            line_width = "dom_by",
                                            fill_color="fi_color")

graph1.node_renderer.hover_glyph = Circle(size="size_dim", fill_alpha="dom_index", 
                                            line_color = "firebrick",
                                            line_dash = [1, 1],
                                            line_width = "dom_by",
                                            fill_color="fi_color")                 
                        
graph1.node_renderer.data_source.data = node_src

graph1.edge_renderer.glyph = MultiLine(line_color="#bdbdbd", line_alpha="width_alpha",
                                               line_dash="dotted", line_width="width")
graph1.edge_renderer.selection_glyph = MultiLine(line_color="edge_col", line_width="width")
graph1.edge_renderer.hover_glyph = MultiLine(line_color="edge_col", line_width="width")
graph1.edge_renderer.data_source.data = edge_src
                        

legend1 = Legend(items=[("Nodes represent sheep, edges represent interactions between them",[]),
                       ("Node Size: Count of Dominance by Percentage of All Sheep Interactions",[]),
                       ("Blue Intensity:  Ratio of Count of Times a sheep dominated vs. was dominated",[]),
                       ("Edge Size: Thicker Represents more Interactions between Sheep",[]),
                       ("Edge Color: Green Represents few, Yellow Medium, Red High Interactions Count",[]),
                       ("Edge Size: Thicker Represents more Interactions between Sheep",[]),
                        ("Circle Border: Thicker Represents more Interactions in which Sheep was Dominated",[]) 
                      ], location="top_right")

graph1.selection_policy = NodesAndLinkedEdges()
graph1.inspection_policy = EdgesAndLinkedNodes()

s1.renderers.append(labels1)
s1.renderers.append(graph1)
s1.add_layout(legend1)

In [31]:
#Second Plot
s2 = figure(x_range=(-1.1,1.1), y_range=(-1.1,1.1),
               title="Step 2: Sheep Domination",tools='pan,wheel_zoom,tap,xbox_select,reset', 
                 width=600, plot_height=600)

s2.add_tools(HoverTool(tooltips=[("Source", "@edges_src"),
                                ("Target", "@edges_tgt"),("Weight","@width")]), 
                        TapTool(), BoxSelectTool(), BoxZoomTool(),WheelZoomTool())

graph2 = from_networkx(X,nx.shell_layout, center=(0,0))

pos = nx.shell_layout(X)
x,y=zip(*pos.values())

src = ColumnDataSource({'x':x,'y':y,'nodeid':nd_idx})
labels2 = LabelSet(x='x', y='y', text='nodeid', source=src)

s2.title.text_color = "black"
s2.title.text_font = "verdana"
s2.title.text_font_style = "italic"
s2.title.text_font_style = "bold"
s2.background_fill_alpha = 0.07
s2.title.align = "center"
s2.title.text_font_size = '15pt'

graph2.node_renderer.glyph = Circle(size="size_dim", fill_alpha="dom_index",
                                            line_color = "firebrick",
                                            line_dash = [1, 1],
                                            line_width = "dom_by",
                                            fill_color="fi_color")

graph2.node_renderer.selection_glyph = Circle(size="size_dim", fill_alpha="dom_index",
                                            line_color = "firebrick",
                                            line_dash = [1, 1],
                                            line_width = "dom_by",
                                            fill_color="fi_color")

graph2.node_renderer.hover_glyph = Circle(size="size_dim", fill_alpha="dom_index", 
                                            line_color = "firebrick",
                                            line_dash = [1, 1],
                                            line_width = "dom_by",
                                            fill_color="fi_color")

graph2.node_renderer.data_source.data = node_src

graph2.edge_renderer.glyph = MultiLine(line_color="#bdbdbd", line_alpha="width_alpha",
                                               line_dash="dotted", line_width="width")

graph2.edge_renderer.selection_glyph = MultiLine(line_color="edge_col", line_width="width")
graph2.edge_renderer.hover_glyph = MultiLine(line_color="edge_col", line_width="width")
graph2.edge_renderer.data_source.data = edge_src

legend2 = Legend(items=[
                       ("Shell Layout for the same graph shown above",[]),
                       ("Please hover on nodes and click edges to see more info",[]) 
                      ], location="top_right")


graph2.selection_policy = NodesAndLinkedEdges()
graph2.inspection_policy = EdgesAndLinkedNodes()

s2.renderers.append(graph2)
s2.renderers.append(labels2)
s2.add_layout(legend2)

In [32]:
##Third Plot
s3 = figure(x_range=s2.x_range, y_range=s2.y_range,
               title="Step 3: Sheep Unique Interactions",tools='pan,wheel_zoom,tap,xbox_select,reset', 
                width=900, plot_height=600)

s3.add_tools(HoverTool(tooltips=[("Source", "@edges_src"),
                                ("Target", "@edges_tgt"),("Weight","@width")]), 
                                TapTool(), BoxSelectTool(), BoxZoomTool(),WheelZoomTool())

graph3 = from_networkx(X,nx.shell_layout, center=(0,0))

s3.title.text_color = "black"
s3.title.text_font = "verdana"
s3.title.text_font_style = "italic"
s3.title.text_font_style = "bold"
s3.background_fill_alpha = 0.07
s3.title.align = "center"
s3.title.text_font_size = '15pt'                                   

graph3.node_renderer.glyph = Rect(x="age", y="age",width="age",height="age",angle=-0.7,
                                            fill_alpha="tgtdivsrcstgt",
                                            line_color = "fi_color3",
                                            line_dash = "solid",
                                            line_width = 5,#"tgtdivsrc",
                                            fill_color="fi_color2")

graph3.node_renderer.selection_glyph = Rect(x="age", y="age",width="age",height="age",angle=-0.7,
                                            fill_alpha="tgtdivsrcstgt",
                                            line_color = "fi_color3",
                                            line_dash = "solid",
                                            line_width = 5,#"tgtdivsrc",
                                            fill_color="fi_color2")

graph3.node_renderer.hover_glyph = Rect(x="age", y="age",width="age",height="age",angle=-0.7,
                                            fill_alpha="tgtdivsrcstgt",
                                            line_color = "fi_color3",
                                            line_dash = "solid",
                                            line_width = 5,#"tgtdivsrc",
                                            fill_color="fi_color2")

graph3.node_renderer.data_source.data = node_src

graph3.edge_renderer.glyph = MultiLine(line_color="#bdbdbd", line_alpha="width_alpha",
                                               line_dash="dotted", line_width="width")
graph3.edge_renderer.selection_glyph = MultiLine(line_color="edge_col", line_width="width")
graph3.edge_renderer.hover_glyph = MultiLine(line_color="edge_col", line_width="width")
graph3.edge_renderer.data_source.data = edge_src


legend3 = Legend(items=[
                       ("Ellipsis Layout for the same graph shown to the left; nodes represented by squares",[]),
                       ("Square Size: sheep age; Square Border Darkness: % of Interactions in which Sheep Was Dominant",[]), 
                       ("Square Color: hover over square to see it colored in green",[]),
                       ("Higher Green intensity implies higher count of interactions with different sheep",[]),   
                       ("Square Border Color: higher intensity of orange implies higher propensity to be dominant during interactions",[]) 
], location="top_right")

graph3.selection_policy = NodesAndLinkedEdges()
graph3.inspection_policy = EdgesAndLinkedNodes()

s3.renderers.append(graph3)
s3.renderers.append(labels2)
s3.add_layout(legend3)

In [33]:
from bokeh.models import LinearColorMapper, ColorBar
from bokeh.palettes import YlOrBr8,Oranges8
O8_inv = Oranges8.reverse()

In [34]:
source = ColumnDataSource(dict(nodes))
color_mapper = LinearColorMapper(palette=Oranges8, low=nodes.WonPCGT.min(), high=nodes.WonPCGT.max())

tls="crosshair,pan,wheel_zoom,box_zoom,reset,hover,previewsave"

s4 = figure(x_axis_label='Age', y_axis_label='Total Interactions with Unique Sheep', 
            title='Step 4: Unique Interactions & Fights Won %',
            tools=tls, 
            toolbar_location='above',
           width=900, plot_height=600)

s4.circle(x='age', y='tot_unique_interactions', color={'field': 'WonPCGT', 'transform': color_mapper}, 
         size=20, alpha=0.6, source=source)

s4.title.text_color = "black"
s4.title.text_font = "verdana"
s4.title.text_font_style = "italic"
s4.title.text_font_style = "bold"
s4.background_fill_alpha = 0.07
s4.title.align = "center"
s4.title.text_font_size = '15pt'

hover =s4.select(dict(type=HoverTool))
hover.tooltips = dict([
    ("Node ID", "@id"),
])

color_bar = ColorBar(color_mapper=color_mapper, label_standoff=12, 
                     location=(0,0), title='Won %',width = 30)

legend4 = Legend(items=[
                       ("Scatter plot representing nodes in terms of Unique Interactions",[]),
                       ("Unique Interactions defined as count of interactions made with different sheep by each sheep ",[]), 
                       ("Scatter points are sheep nodes. Point color reflects square border color from graph below",[]),
                       ("Sheep with higher chances of dominating interactions depicted in darker shades of orange",[]),   
                       ("Conclusion: sheep that have interactions with a wider variety of sheep have a higher chance of being dominant",[]), 
], location="top_right")


s4.add_layout(legend4)
s4.add_layout(color_bar, 'right')

In [35]:
p = gridplot([[s1,s4], [s2,s3]])

show(p)

E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name: age, fi_color2, fi_color3, tgtdivsrcstgt [renderer: GlyphRenderer(id='7175a127-8595-45bf-9875-19fa91f0b4e7', ...)]
E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name: dom_by, dom_index, fi_color, size_dim [renderer: GlyphRenderer(id='9db2adda-60b2-4998-8089-b28811262089', ...)]
E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name: dom_by, dom_index, fi_color, size_dim [renderer: GlyphRenderer(id='d669a1d3-6c65-4115-8905-218c5f4777b5', ...)]
E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name: width, width_alpha [renderer: GlyphRenderer(id='3191c56c-2f64-4fc5-8432-a3915373dd08', ...)]
E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name: width, width_alpha [renderer: GlyphRenderer(id='87657622-a167-46ef-a43e-c52402e0d1d8', ...)]
E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name: width, width_alpha [renderer: GlyphRenderer(id='f3b2f123-6408-4500-90e7-f